In [1]:
import os
import shutil
import psi4
from rdkit import Chem
from rdkit.Chem import AllChem

scratch_dir = os.path.join(os.getcwd(), 'psi4_scratch')
if os.path.exists(scratch_dir):
    shutil.rmtree(scratch_dir)
os.makedirs(scratch_dir)
os.environ['PSI_SCRATCH'] = scratch_dir

psi4.set_memory('2 GB')
psi4.set_num_threads(4)
psi4.core.set_output_file('psi4_output.dat', False)

def gerar_geometria_3d(smiles, nome):
    print(f"--- Gerando 3D inicial (MMFF) para {nome}... ---")
    mol = Chem.MolFromSmiles(smiles)
    mol = Chem.AddHs(mol)
    AllChem.EmbedMolecule(mol, randomSeed=42)
    AllChem.MMFFOptimizeMolecule(mol)
    return Chem.MolToXYZBlock(mol)

smiles_sacarose = "O[C@H]1[C@H](O)[C@@H](CO)O[C@@]1(CO)O[C@@H]2[C@H](O)[C@@H](O)[C@H](O)[C@@H]2CO"
smiles_sucralose = "ClC[C@H]1O[C@](CCl)(O[C@@H]2O[C@H](CO)[C@H](O)[C@H]2Cl)[C@@H](O)[C@@H]1O"

geo_sacarose = gerar_geometria_3d(smiles_sacarose, "Sacarose")
geo_sucralose = gerar_geometria_3d(smiles_sucralose, "Sucralose")

def otimizar_molecula_leve(xyz_block, label):
    """
    A geometria do RDKit (MMFF) já é 90% boa para visualização.
    """
    print(f"\n[{label}] >>> Iniciando Cálculo SCF (B3LYP/3-21G)...")
    
    psi4.core.clean()
    mol = psi4.geometry(xyz_block)
    
    psi4.set_options({
        'basis': '3-21G',
        'reference': 'rks',
        'scf_type': 'df',
        'soscf': 'true'
    })
    
    try:
        print(f"[{label}] Modo Seguro: Calculando Energia (Single Point)...")
        e, wfn = psi4.energy('b3lyp', return_wfn=True)
        
        print(f"[{label}] Sucesso! Energia: {e:.4f} Ha")
        return wfn
        
    except Exception as e:
        print(f"[{label}] Erro no cálculo: {e}")
        return None

wfn_sacarose = otimizar_molecula_leve(geo_sacarose, "Sacarose")
wfn_sucralose = otimizar_molecula_leve(geo_sucralose, "Sucralose")

def gerar_cubes(wfn, label):
    if wfn:
        print(f"[{label}] Gerando arquivos .cube...")
        psi4.set_options({
            'cubeprop_tasks': ['esp', 'density'],
            'cubic_grid_spacing': [0.2, 0.2, 0.2],
            'cubic_grid_overage': [4.0, 4.0, 4.0] 
        })
        psi4.cubeprop(wfn)
        
        import os
        base_esp = f"{label}_ESP.cube"
        base_dens = f"{label}_Density.cube"
        
        if os.path.exists("ESP.cube"): os.rename("ESP.cube", base_esp)
        if os.path.exists("Dt.cube"): os.rename("Dt.cube", base_dens)
        print(f"[{label}] Arquivos {base_esp} e {base_dens} salvos!")

gerar_cubes(wfn_sacarose, "Sacarose")
gerar_cubes(wfn_sucralose, "Sucralose")

--- Gerando 3D inicial (MMFF) para Sacarose... ---
--- Gerando 3D inicial (MMFF) para Sucralose... ---

[Sacarose] >>> Iniciando Cálculo SCF (B3LYP/3-21G)...
[Sacarose] Modo Seguro: Calculando Energia (Single Point)...
[Sacarose] Sucesso! Energia: -1215.9255 Ha

[Sucralose] >>> Iniciando Cálculo SCF (B3LYP/3-21G)...
[Sucralose] Modo Seguro: Calculando Energia (Single Point)...
[Sucralose] Sucesso! Energia: -2324.6207 Ha
[Sacarose] Gerando arquivos .cube...
[Sacarose] Arquivos Sacarose_ESP.cube e Sacarose_Density.cube salvos!
[Sucralose] Gerando arquivos .cube...
[Sucralose] Arquivos Sucralose_ESP.cube e Sucralose_Density.cube salvos!
